In [135]:
import os, glob
import pandas as pd
import subprocess as sp
from IPython.core import display as ICD 
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)



In [136]:
scan_file ="bbx_scan_QC.tsv"

In [137]:
scan_df = pd.read_csv(scan_file,sep="\t")

In [138]:
scan_df.columns.values

array(['Unnamed: 0', 'BIDS check', 'fmriprep check', 'Anat', 'RUN 1',
       'Unnamed: 5', 'RUN 2', 'Unnamed: 7', 'RUN 3', 'Unnamed: 9',
       'RUN 4', 'Unnamed: 11', 'From TSV Files', 'Run 1', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Run 2', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Run 3', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Run 4', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'FSL Level 1', 'Run 1.1', 'Run 2.1',
       'Run 3.1', 'Run 4.1', 'FINAL CHECK', 'Unnamed: 35'], dtype=object)

In [139]:
scan_df.columns = ['subject', 'BIDS check', 'fmriprep check', 'Anat', 'epi_r1',
       'sdc_r1', 'epi_r2', 'sdc_r2', 'epi_r3', 'sdc_r3',
       'epi_r4', 'sdc_r4', 'From TSV Files', 'fd_r1', 'min_r1',
       'max_r1', 'mcflirt_3mm_r1', 'fd_r2', 'min_r2',
       'min_r2', 'mcflirt_3mm_r2', 'fd_r3', 'min_r3',
       'max_r3', 'mcflirt_3mm_r3', 'fd_r4', 'min_r4',
       'max_r4', 'mcflirt_3mm_r4', 'FSL Level 1', 'activation_r1', 'activation_r2',
       'activation_r3', 'activation_r4', 'FINAL CHECK', 'Intials']

In [140]:
col_details = scan_df.iloc[0:2, :]
col_details['fd_r1']

0    FD Check (timepoints >0.9)
1    NaN                       
Name: fd_r1, dtype: object

In [141]:
scan_df.drop([0,1], inplace=True)

In [142]:
scan_df = scan_df.set_index(["subject"])


In [143]:
scan_df.head(65)

,BIDS check,fmriprep check,Anat,epi_r1,sdc_r1,epi_r2,sdc_r2,epi_r3,sdc_r3,epi_r4,...,min_r4,max_r4,mcflirt_3mm_r4,FSL Level 1,activation_r1,activation_r2,activation_r3,activation_r4,FINAL CHECK,Intials
subject,,,,,,,,,,,,,,,,,,,,,
sub-001,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-002,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-003,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-004,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-005,y,--,good,good,good,good,good,good,good,good,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-061,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-062,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN
sub-063,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
bids_path = '/projects/niblab/bids_projects/Experiments/bbx/bids'
# FD check
sub_ids = [x.split("/")[-2] for x in glob.glob(os.path.join(bids_path, 'derivatives/fmriprep/sub-*/ses-1'))]
for sub_id in sub_ids:
    tsv_files=glob.glob(os.path.join(bids_path, 'derivatives/fmriprep/{}/ses-1/func/sub-*_ses-1_task-training_run-*_desc-confounds_regressors.tsv'.format(sub_id)))
    
    for tsv in sorted(tsv_files):
        fd_total_ct = 0
        sub_id=tsv.split("/")[-1].split("_")[0]
        #print(tsv)
        run_id=tsv.split("_")[4]
        nifti=os.path.join(bids_path,
                           'derivatives/fmriprep/{}/ses-1/func/{}_ses-1_task-training_{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz '.format(sub_id, sub_id, run_id))
        shell_cmd1 = 'fslnvols {}'.format(nifti) 
        volume = sp.check_output(shell_cmd1, shell=True, encoding="utf-8")
        volume = volume.strip()
        comparator = int(volume) *.25
        temp_df = pd.read_csv(tsv, sep="\t")
        
        
        for fd_val in  temp_df['framewise_displacement']:
            if fd_val > 0.9:
                #print("bad fd value, ", fd_val)
                fd_total_ct += 1
        
        if fd_total_ct < comparator:
            if run_id == "run-1":
                scan_df.loc[sub_id, "fd_r1"]="pass"
            elif run_id == "run-2":
                scan_df.loc[sub_id, "fd_r2"]="pass"
            elif run_id == "run-3":
                scan_df.loc[sub_id, "fd_r3"]="pass"
            else:
                scan_df.loc[sub_id, "fd_r4"]="pass"
                
        else:
            if run_id == "run-1":
                scan_df.loc[sub_id, "fd_r1"]="fail"
            elif run_id == "run-2":
                scan_df.loc[sub_id, "fd_r2"]="fail"
            elif run_id == "run-3":
                scan_df.loc[sub_id, "fd_r3"]="fail"
            else:
                scan_df.loc[sub_id, "fd_r4"]="fail"
            
            print("sub: {} \trun: {}  \tcomparator: {} \tfd_count: {} \nNifti: {}".format(sub_id,
                                         run_id, comparator, fd_total_ct, nifti.split("/")[-1]))
    


sub: sub-070 	run: run-1  	comparator: 58.25 	fd_count: 72 
Nifti: sub-070_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-100 	run: run-1  	comparator: 0.0 	fd_count: 1 
Nifti: sub-100_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-100 	run: run-2  	comparator: 0.0 	fd_count: 0 
Nifti: sub-100_ses-1_task-training_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-100 	run: run-3  	comparator: 0.0 	fd_count: 0 
Nifti: sub-100_ses-1_task-training_run-3_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-100 	run: run-4  	comparator: 0.0 	fd_count: 2 
Nifti: sub-100_ses-1_task-training_run-4_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-098 	run: run-1  	comparator: 0.0 	fd_count: 0 
Nifti: sub-098_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz 
sub: sub-098 	run: run-2  	comparator: 0.0 	fd_count: 0 
Nifti: sub-098_ses-1_task-training_run-2

subject
sub-001    pass
sub-002    pass
sub-003    NaN 
sub-004    pass
sub-005    pass
sub-006    pass
sub-007    pass
sub-008    NaN 
sub-009    pass
sub-010    pass
sub-011    NaN 
sub-012    NaN 
sub-013    NaN 
sub-014    NaN 
sub-015    pass
sub-016    NaN 
sub-017    NaN 
sub-018    NaN 
sub-019    NaN 
sub-020    NaN 
sub-021    pass
sub-022    pass
sub-023    NaN 
sub-024    NaN 
sub-025    NaN 
sub-026    pass
sub-027    pass
sub-028    pass
sub-029    NaN 
sub-030    pass
sub-031    pass
sub-032    pass
sub-033    NaN 
sub-034    pass
sub-035    NaN 
sub-036    pass
sub-037    NaN 
sub-038    NaN 
sub-039    pass
sub-040    pass
sub-041    pass
sub-042    pass
sub-043    pass
sub-044    pass
sub-045    pass
sub-046    pass
sub-047    NaN 
sub-048    NaN 
sub-049    NaN 
sub-050    pass
Name: fd_r4, dtype: object